# Output Parser
**Output Parser**는 대규모 언어 모델(LLM, Large Language Model)의 출력 결과를 **애플리케이션에서 활용할 수 있도록 적절한 형식으로 변환**하는 도구이다.
- LLM은 일반적으로 텍스트 형태로 응답을 생성하지만, 이 텍스트는 그대로 활용하기 어려운 경우가 많다.
- Output Parser는 이러한 **비구조적 텍스트 데이터를 구조화된 데이터로 변환**하여 프로그램에서 활용 가능하도록 만든다.
- 예를 들어, 키워드 리스트를 뽑거나 JSON 형식으로 정보를 변환하는 데 사용된다.

## 주요 Output Parser 종류

1. **CommaSeparatedListOutputParser**
   - 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
   - 예: `"사과, 바나나, 포도"` → `["사과", "바나나", "포도"]`
2. **JsonOutputParser**
   - LLM의 출력이 JSON 형식일 때 이를 Python의 `dict` 객체로 변환한다.
   - JSON(JavaScript Object Notation)은 데이터 구조를 표현하기 위한 경량 포맷이다.
3. **PydanticOutputParser**
   - JSON 데이터를 Python의 [Pydantic](https://docs.pydantic.dev) 모델로 변환한다.
   - Pydantic은 데이터 유효성 검사와 설정 관리에 널리 사용되는 Python 라이브러리이다.
4. **StrOutputParser**
   - 모델의 출력 결과를 단순 문자열로 반환한다.
   - Chat 기반 모델은 Message 객체의 속성으로 LLM 결과를 반환한다. 거기에서 응답 문자열만 추출해서 반환한다.
> `JsonOutputParser`, `PydanticOutputParser` 는 모두 Pydantic을 사용해 데이터 구조(schema)를 정의하고, 해당 구조에 따라 출력을 검증하고 변환한다.

## 주요 메소드
- `parse(text: str)`
  - LLM이 생성한 문자열 응답을 받아 정해진 구조로 변환하여 반환한다.
- `get_format_instructions() -> str`
  - 각 OutputParer가 변환할 수있는 형식으로 LLM이 응답하도록 하는 프롬프트 텍스트를 반환한다.
  - 이 내용을 프롬프트에 넣어서 LLM이 정확한 포맷으로 응답하도록 유도한다.
  
## 참고
- Output Parser는 일반적으로 [`Runnable`](05_chaing_LECL.ipynb#Runnable) 인터페이스를 상속하여 구현되며, `invoke()` 메서드를 통해 실행할 수 있다.
- `invoke()`는 내부적으로 `parse()`를 호출하여 동작한다.
- 필요한 경우 Output Parser를 직접 구현하여 사용자 정의 출력 포맷을 처리할 수도 있다. 


## StrOutputParser
- 모델(LLM)의 출력 결과를 string으로 변환하여 반환하는 output parser.
- Chat Model은  Message 객체에서 content 속성값을 추출하여 문자열로 반환한다.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 단순 질의용 prompt 생성-from_template() 메소드 이용. (PromptTemplate용)
prompt = ChatPromptTemplate.from_template(
    template="한국의 {topic}에 관련된 속담 {count}개를 알려줘. 목록 형식으로 출력해줘."
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()
# prompt -> model -> parser

query = prompt.invoke({"topic":"호랑이", "count":3})
res = model.invoke(query)
final_res = parser.invoke(res)

In [ ]:
res

AIMessage(content='1. 호랑이도 제 말하면 온다 — 어떤 사람을 이야기하면 그 사람이 실제로 나타날 때 쓰는 말(영어의 "speak of the devil"와 유사).  \n2. 호랑이 굴에 들어가야 호랑이 새끼를 잡는다 — 큰 성과를 얻으려면 위험을 감수해야 한다는 뜻.  \n3. 범에게 물려가도 정신만 차리면 산다 — 위험한 상황에서도 침착하면 위기를 벗어날 수 있다는 뜻.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 955, 'prompt_tokens': 30, 'total_tokens': 985, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 832, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CnbHElKxhzQzw9gZ2y08COPRCPuWW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2a12-4586-7bd0-b308-76bf95c73de5-0', usage_metadata={'input_tokens': 30, 'output_tokens': 955, 'total_tokens': 985, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'au

In [6]:
print(final_res)

1. 호랑이도 제 말하면 온다 — 어떤 사람을 이야기하면 그 사람이 실제로 나타날 때 쓰는 말(영어의 "speak of the devil"와 유사).  
2. 호랑이 굴에 들어가야 호랑이 새끼를 잡는다 — 큰 성과를 얻으려면 위험을 감수해야 한다는 뜻.  
3. 범에게 물려가도 정신만 차리면 산다 — 위험한 상황에서도 침착하면 위기를 벗어날 수 있다는 뜻.


## CommaSeparatedListOutputParser

- 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
  - "a,b,c" => ['a','b','c']

In [10]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
txt = "이순신,유관순,안중근,강감찬,세종대왕"
txt = "요청하신 이름은 이순신과 유관순과 안중근입니다."
r1 = parser.parse(txt)
r2 = parser.invoke(txt)
print(type(r1), type(r2))

<class 'list'> <class 'list'>


In [11]:
print(r1)
print(r2)

['요청하신 이름은 이순신과 유관순과 안중근입니다.']
['요청하신 이름은 이순신과 유관순과 안중근입니다.']


In [ ]:
# 이 output parser에 맞는 출력을 모델이 하도록 하는 지시문으로 프롬프트에 추가한다.
parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [27]:
parser = CommaSeparatedListOutputParser()
prompt = ChatPromptTemplate(
    messages=[
        {"role":"system", "content": "Output Format: {format_instruction}"},
        {"role":"user", "content":"{subject}에 대해 다섯가지를 나열해 주세요."}
    ],
    partial_variables={"format_instruction":parser.get_format_instructions()}
)
# partial_variables={"input_variable":넣을 값,} => input_variable의 값을 Prompt Template을 
#   만들면서 넣어준다. (invoke 시 넣는 것이 아니라 생성시 설정.)
model = ChatOpenAI(model="gpt-5-mini")

query = prompt.invoke({"subject":"자동차 종류"})
res = model.invoke(query)

In [28]:
print(res.content)
final_res = parser.invoke(res)
final_res

세단, SUV, 해치백, 쿠페, 컨버터블


['세단', 'SUV', '해치백', '쿠페', '컨버터블']

In [29]:
# 질의 ->(Prompt)->query->(Model)->res->(Parser)->최종답변
# Chain: 위 실행흐름을 자동화 (pipeline)
chain = prompt | model | parser
res = chain.invoke({"subject":"오픈소스 LLM 모델 이름"})
print(res)

['Llama 2', 'Falcon 40B', 'MPT-7B', 'BLOOM', 'GPT-NeoX-20B']


## JsonOutputParser

- JSON 형식의 응답을 dictionary로 반환한다.
- JSON 형식을 정하려는 경우 [Pydantic](Ref_typing_Pydantic.ipynb)을 이용해 JSON 스키마를 정의하여 JsonOutputParser 생성시 전달한다.
  - Pydantic 모델클래스를 이용해 LLM 모델이 응답할 때 json의 어떤 key에 어떤 응답을 작성할 지 Field로 정의한다.
  - Schema 지정은 필수는 아니다. 
- LLM이 JSON Schema를 따르는 형태로 응답을 하면 JsonOutputParser는 Dictionary로 변환한다.

In [38]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
print(parser.get_format_instructions())

txt = '{"name":"홍길동","age":20}'
# txt = "이름: 홍길동, 나이: 20"
r = parser.invoke(txt)
print(type(r))
r['age']

Return a JSON object.
<class 'dict'>


20

In [63]:
# JSON 응답 스키마(구조 설계) 정의 -> Pydantic
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name: str = Field(description="조회한 사람의 이름")
    yob: int = Field(description="조회한 사람이 태어난 년도")
    yod: int = Field(description="조회한 사람이 사망한 년도")
    profile: str = Field(description="조회한 사람의 주요 업적")

# 변수: key 
# Field.description: 변수에 넣을 내용

In [59]:
parser2 = JsonOutputParser(pydantic_object=PersonInfo)
print(parser2.get_format_instructions())

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown in your output):


In [60]:
# parser = JsonOutputParser()
parser = JsonOutputParser(pydantic_object=PersonInfo)
prompt = ChatPromptTemplate(
    [
        {"role":"system", "content":"Output Format: {output_format}"},
        {"role":"user", "content":"{query}"}
    ],
    partial_variables={"output_format":parser.get_format_instructions()}
)
model = ChatOpenAI(model='gpt-5-mini')

query = prompt.invoke({"query":"이순신 장군에 대해서 알려줘."})
res = model.invoke(query)

In [61]:
final_res = parser.invoke(res)

In [62]:
print(type(res.content), type(final_res))
final_res

<class 'str'> <class 'dict'>


{'name': '이순신',
 'yob': 1545,
 'yod': 1598,
 'profile': '조선 중기의 무신이자 명장. 임진왜란(1592-1598) 동안 조선 수군을 지휘하여 다수의 해전에서 승리했고, 거북선과 학익진 전술을 활용해 해상 우위를 확보했다. 한산도 대첩, 명량 해전(13척으로 다수의 적을 격파), 노량 해전에서 전사(1598) 등으로 조선을 지키는 데 결정적 기여를 했으며, 충무공이라는 시호로 추앙받는다.'}

## PydanticOutputParser

- JSON 형태로 받은 응답을 Pydantic 모델로 변환하여 반환한다.
- 구현은 JsonOutputParser와 동일한데 parsing 결과를 pydantic 모델타입으로 반환한다.

In [65]:
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name: str = Field(description="조회한 사람의 이름")
    yob: int = Field(description="조회한 사람이 태어난 년도")
    yod: int = Field(description="조회한 사람이 사망한 년도")
    profile: str = Field(description="조회한 사람의 주요 업적")

In [66]:
from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=PersonInfo)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "조회한 사람의 이름", "title": "Name", "type": "string"}, "yob": {"description": "조회한 사람이 태어난 년도", "title": "Yob", "type": "integer"}, "yod": {"description": "조회한 사람이 사망한 년도", "title": "Yod", "type": "integer"}, "profile": {"description": "조회한 사람의 주요 업적", "title": "Profile", "type": "string"}}, "required": ["name", "yob", "yod", "profile"]}
```


In [67]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=PersonInfo)

prompt = ChatPromptTemplate(
    [
        {"role":"system", "content":"Output Format: {output_format}"},
        {"role":"user", "content":"{query}"}
    ],
    partial_variables={"output_format":parser.get_format_instructions()}
)
model = ChatOpenAI(model='gpt-5-mini')

query = prompt.invoke({"query":"이순신 장군에 대해서 알려줘."})
res = model.invoke(query)

In [ ]:
res

AIMessage(content='{"name":"이순신","yob":1545,"yod":1598,"profile":"조선 중기의 해군 장군. 임진왜란(1592–1598) 동안 탁월한 지휘로 한산도 대첩(1592)·명량 해전(1597) 등에서 결정적 승리를 거두었으며 거북선과 학익진 등 전술을 활용해 일본 수군을 격파했다. 난중일기 등 기록을 남겼고, 1598년 노량해전에서 전사했으며 사후 충무공으로 추서되었다."}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 721, 'prompt_tokens': 274, 'total_tokens': 995, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cnccw7En0WJJYl1VHvrjWXifGwJf7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2a61-7a9a-7910-92ac-4d7708e91a2c-0', usage_metadata={'input_tokens': 274, 'output_tokens': 721, 'total_tokens': 995, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token

In [69]:
final_res = parser.invoke(res)
type(final_res)

__main__.PersonInfo

In [70]:
final_res

PersonInfo(name='이순신', yob=1545, yod=1598, profile='조선 중기의 해군 장군. 임진왜란(1592–1598) 동안 탁월한 지휘로 한산도 대첩(1592)·명량 해전(1597) 등에서 결정적 승리를 거두었으며 거북선과 학익진 등 전술을 활용해 일본 수군을 격파했다. 난중일기 등 기록을 남겼고, 1598년 노량해전에서 전사했으며 사후 충무공으로 추서되었다.')

In [72]:
final_res.name
final_res.profile

'조선 중기의 해군 장군. 임진왜란(1592–1598) 동안 탁월한 지휘로 한산도 대첩(1592)·명량 해전(1597) 등에서 결정적 승리를 거두었으며 거북선과 학익진 등 전술을 활용해 일본 수군을 격파했다. 난중일기 등 기록을 남겼고, 1598년 노량해전에서 전사했으며 사후 충무공으로 추서되었다.'

# LLM모델에 출력 형식을 설정

- ChatModel객체의 `with_structured_output(pydantic.BaseModel)` 을 이용해 모델의 출력 형식을 모델 자체에 추가할 수있다.
- `OutputParser`는 모델의 출력 결과를 받아서 형식을 변경해 준다. 그래서 Chain에 탈/부착을 통해 형식을 적용하거나 적용하지 않는 것을 자유롭게 할 수있다.
- 모델의 출력 결과를 항상 일정하게 할 경우에는 아예 **모델에 출력 형식을 설정할 수 있다.**

In [4]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# 출력 schema 정의
class OutputSchmea(BaseModel):
    is_real_person : bool = Field(...,description='요청한 인물이 실존 인물인지 여부. True:신존인물, False:신졸인물이 아님.')
    description: str = Field(...,description='요청한 인물에 대한 설명')
    
    model = ChatOpenAI(model='gpt-5-mini')
    res = model.invoke('이순신 장군에 대해 설명해줘.')


PydanticUserError: A non-annotated attribute was detected: `model = ChatOpenAI(profile={'max_input_tokens': 400000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x0000026A3E019C70>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000026A3E057500>, root_client=<openai.OpenAI object at 0x0000026A3C983BC0>, root_async_client=<openai.AsyncOpenAI object at 0x0000026A3C84C9E0>, model_name='gpt-5-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)`. All model fields require a type annotation; if `model` is not meant to be a field, 

In [ ]:
print(type(res))
print(res.content)

In [ ]:
model_with_output = model.with_structured_output(OoutputSchema)
res2 = model_with_output.invoke('이순신 장군에 대해 설명해줘')

In [ ]:
print(type(res2))
print(res2)
print(res2.is_real_person)
print(res2.description)

# Streaming 방식 응답 처리

- Streaming 방식 응답 처리란, LLM이 텍스트를 모두 생성할 때까지 기다리지 않고, 생성되는 즉시 **부분적인 결과**를 실시간으로 전달받아 처리하는 방식을 의미한다. 이는 사용자가 응답을 더 빠르게 인지할 수 있게 해 주며, 특히 대화형 서비스, 실시간 UI 출력, 긴 문서 생성과 같은 상황에서 매우 유용하게 활용된다.

- `invoke()` 요청으로 받는 응답은 **비 스트리밍 방식**으로 모든 응답 텍스트 생성이 완료된 이후 그 결과를 한 번에 반환하는 구조이다. 반면 Streaming 방식은 **토큰(token) 단위 또는 여러 토큰이 묶인 청크(chunk) 단위**로 연속적인 데이터 스트림을 전송한다는 점에서 큰 차이가 있다. 즉, Streaming은 마치 사람이 타이핑을 치듯이 응답을 실시간으로 “흘려보내는” 방식이라고 이해할 수 있다.

- `모델.invoke(input, config)` → 응답 데이터
    - 모델이 전체 응답을 모두 생성한 뒤, 최종 결과를 한 번에 반환하는 방식이다.
    - 배치 처리나 후처리가 중요한 경우에 적합하다.
- `모델.stream(input, config)` → Iterator
    - 모델이 토큰을 생성하는 즉시, 순차적으로 결과를 제공하는 Iterator 형태로 반환한다.
    - 실시간 출력, 대화형 인터페이스, 웹 스트리밍 등에 특히 적합하다.

In [5]:
from langchain_openai import ChatOpenAI

In [6]:
model = ChatOpenAI(model='gpt-5-mini')#, streaming=True)
iterator = model.stream('서울의 유명한 관광지를 소개해줘. 추천 이유도 알려줘.')
print(type(iterator))
for token in iterator:
    print(token)

<class 'generator'>
content='' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-e5d5-7452-8d26-873d30cd6430'
content='서울' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-e5d5-7452-8d26-873d30cd6430'
content='의' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-e5d5-7452-8d26-873d30cd6430'
content=' 유명' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-e5d5-7452-8d26-873d30cd6430'
content='한' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-e5d5-7452-8d26-873d30cd6430'
content=' 관광' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-e5d5-7452-8d26-873d30cd6430'
content='지를' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-e5d5-7452-8d26-873d30cd6430'
content=' 분야' additional_kwargs={} response_metadata={'model_prov

In [7]:
for token in model.stream('서울의 유명한 관광지를 소개해줘. 추천 이유도 알려줘.'):
    print(token.content, end='')

아래는 서울의 대표 관광지들을 관심사별로 정리한 목록과 추천 이유(간단한 방문 팁 포함)입니다.

역사·전통
- 경복궁: 조선의 중심 궁궐로 건축과 왕실 문화 체험 가능. 광화문·근정전 등 포인트가 많아 사진 명소. (한복 착용 시 무료 입장 혜택(일부 경우) / 오전 방문 추천)
- 창덕궁·후원(비원): 조선 시대 궁궐과 자연이 어우러진 비밀정원. 가이드 투어로 후원 입장 가능해 고요한 산책에 좋음.
- 북촌한옥마을: 전통 한옥들이 밀집한 동네로 고즈넉한 풍경과 골목 산책에 최적. 카페·공방도 많음(주거지역이라 예의 필수).

문화·예술·전통체험
- 인사동: 전통 공예품, 갤러리, 전통 찻집이 모여 있는 문화거리. 기념품·한지·도자기 쇼핑 좋음.
- 국립중앙박물관: 한국 역사·예술을 폭넓게 전시. 규모가 커서 비 오는 날 방문 추천(주요 전시 무료 또는 저렴).

쇼핑·먹거리·시장
- 명동: 화장품·패션 숍이 밀집한 쇼핑 중심지. 길거리 음식과 외국인 관광객 서비스가 발달해 초행자에게 편리.
- 광장시장: 전통시장 먹거리(빈대떡, 마약김밥 등)와 한복·직물 상가. 한국 길거리 음식 체험에 최적.

야경·전망·한강
- N서울타워(남산): 서울 전경을 한눈에 볼 수 있는 전망대. 야경과 데이트 코스로 인기(저녁·일몰 시간 추천).
- 한강공원(여의도·반포 등): 강변에서 피크닉, 자전거, 한강 유람선, 반포대교 달빛무지개분수 등 레저 활동 가능.

젊음·예술·밤문화
- 홍대(홍익대 주변): 젊음의 거리, 스트리트 공연·클럽·개성 있는 카페·플리마켓 등 창의적인 분위기.
- 이태원: 다문화 음식·바·외국인 친화적인 분위기. 밤문화와 다양한 국제 요리를 즐기기 좋은 곳.

건축·야간 쇼핑
- 동대문디자인플라자(DDP): 독특한 건축물(자하 하디드 설계)과 야간시장, 디자인 전시가 열리는 공간. 패션 도매·야시장도 유명.

가족·테마파크
- 롯데월드·롯데월드타워(서울스카이): 실내·실외 놀이시설과 전망대, 대형 쇼핑몰과 수족관 등 가족 단위 인기 코스.

도심 속 휴